In [1]:
import pandas as pd
import psycopg2
import config
import workdays
import datetime



holidays_br = workdays.load_holidays('BR')
holidays_b3 = workdays.load_holidays('B3')

dt = datetime.date.today()
vcto_0 = dt.strftime('%d/%m/%Y')

dt_pos = workdays.workday(dt, -1, holidays_br)
dt_1 = workdays.workday(dt, -1, holidays_b3)
dt_2 = workdays.workday(dt, -2, holidays_b3)
dt_3 = workdays.workday(dt, -3, holidays_b3)



dt


datetime.date(2022, 12, 29)

In [2]:
dt

datetime.date(2022, 12, 29)

In [3]:
# ## d-1

def get_carteira_pos(mercado,codigo,dt,serie=None):
    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)
    if serie!=None:
        query=f"SELECT  * FROM public.tbl_carteira1 where str_mercado='{mercado}'  and str_mesa='Kapitalo 11.1' and dte_data='{dt}' and str_serie='{serie}' \
                "
    else:
        query=f"SELECT  * FROM public.tbl_carteira1 where str_mercado='{mercado}'  and str_mesa='Kapitalo 11.1' and dte_data='{dt}'"
    df =pd.read_sql(query,db_conn_test)
    db_conn_test.close()
    
    return df

def get_carteira_estrategia(dt,estrategia):
    
    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)
    query=f"SELECT  * FROM public.tbl_carteira1 where str_mesa='Kapitalo 11.1' and dte_data='{dt}' and str_estrategia = '{estrategia}'"
    df =pd.read_sql(query,db_conn_test)
    
    db_conn_test.close()
    return df
    
def get_ibotz_trades_offshore(d=None):
    if d is None:
        d = workdays.workday(datetime.date.today(), 0)

    d_1 = workdays.workday(d, days=0, holidays=workdays.load_holidays("B3"))

    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)
    query = f"SELECT * FROM public.tbl_boletasexternas WHERE (str_mesa='Kapitalo 11.1') and dte_data='{d_1.strftime('%Y-%m-%d')}'"

    df = pd.read_sql(query, db_conn_test)
    db_conn_test.close()
    return df

def get_ibotz_trades(d=None):
    if d is None:
        d = workdays.workday(datetime.date.today(), 0)
    db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)
    d_1 = workdays.workday(d, days=0, holidays=workdays.load_holidays("B3"))


    query = f"SELECT * FROM tbl_auxboletas1 WHERE (str_mesa='Kapitalo 11.1' or (str_fundo='KAPITALO KAPPA MASTER FIM')) and dte_data='{d_1.strftime('%Y-%m-%d')}'"

    df = pd.read_sql(query, db_conn_test)
    db_conn_test.close()
    return df

## Controle  S&P

In [4]:
def get_posicao(mercado,codigo,dt,serie):
    df = get_carteira_pos(mercado,codigo,dt,serie)
    

    df = (
        df.groupby(
            ["str_fundo", "str_codigo", "str_serie", "str_estrategia"], as_index=False
        )
        .agg({"dbl_lote": sum})
        .reset_index()
    )
    df.rename(columns={"dbl_lote": "position"}, inplace=True)

    df_trades_offshore = get_ibotz_trades_offshore()
    df_trades_offshore = df_trades_offshore[df_trades_offshore['str_mercado'] == 'Futuro']
    df_trades_offshore = df_trades_offshore.groupby(['str_fundo', 'str_serie', 'str_codigo', 'str_estrategia'], as_index=False).agg({"dbl_qtetrade": sum}).reset_index()
    df_trades_offshore.rename(columns={'dbl_qtetrade': 'trades'}, inplace=True)
    df_trades_offshore['str_fundo'] = df_trades_offshore['str_fundo'].map({
        'KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO B USD': 'KAPITALO CLASS B',
        'KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO Q USD': 'KAPITALO CLASS Q',
        'KAPITALO INTERNATIONAL FUND SPC - SEGREGATED PORTFOLIO K USD': 'KAPITALO CLASS K',

    })

    df_trades = get_ibotz_trades()
    
    df_trades = df_trades[df_trades["str_mercado"] == "Futuro"]
    df_trades = df_trades[df_trades['str_codigo'].isin(['WSP','ISP','ES','MES@CME'])]
    
    if not df_trades.empty:
        df_trades.rename(columns={"dbl_lote": "trades"}, inplace=True)
        
        roll = df_trades[df_trades['str_serie'].str.match('[A-Z]\d[A-Z]\d')]
        if not roll.empty:
            year = str(datetime.date.today().year)[-1]
            roll['leg'] = roll['trades'].apply(lambda x: ['short', 'long'])
            roll = roll.explode('leg')
            roll['str_serie'] = roll.apply(lambda row: str(row['str_serie'][:2]) + year if row['leg'] == 'short' else str(row['str_serie'][2:]) + year, axis=1)
            roll['trades'] = roll.apply(lambda row: -row['trades'] if row['leg'] == 'short' else row['trades'], axis=1)
            roll['str_codigo'] = roll['str_codigo'].map({
                'RSP': 'ISP',
                'WS1': 'WSP',
                'WI1':'WIN',
                'IR1':'IND'
            })
            df_trades = pd.concat([df_trades[~df_trades['str_serie'].str.match('[A-Z]\d[A-Z]\d')], roll])
        
        df_trades = (
            df_trades.groupby(
                ["str_fundo", "str_serie", "str_codigo", "str_estrategia"], as_index=False
            )
            .agg({"trades": sum})
            .reset_index()
        )
        
        df_trades = pd.concat([df_trades, df_trades_offshore])
        
        
        df = df.merge(
            df_trades[["str_fundo", "str_codigo",
                    "trades", "str_serie", "str_estrategia"]],
            on=["str_fundo", "str_codigo", "str_serie", "str_estrategia"],
            how="left",
        )
    try:    
        df["trades"].fillna(0, inplace=True)
    except:
        df['trades']=0
    df["net_position"] = df["position"] + df["trades"]
    df = df.drop("index", axis=1)
     
    return df


In [8]:
off_fut=get_posicao(mercado='Futuro',codigo=None,dt=dt_1,serie="H23")
off_fut.rename(columns={'net_position':'dbl_lote'},inplace=True)
off_fut=off_fut[off_fut['str_fundo'].isin(['KAPITALO CLASS B', 'KAPITALO CLASS Q', 'KAPITALO CLASS K'])]
off_fut=off_fut[~off_fut['str_codigo'].isin(['TY10','ZN'])]

off_fut['dbl_lote']=off_fut.apply(lambda row: (row['position']*10) if row['str_codigo']=='ES' else row['position'] if row['str_codigo'] in ['ES','MES@CME'] else 0,axis=1)
off_fut['trades']=off_fut.apply(lambda row: (row['trades']*10) if row['str_codigo']=='ES' else row['trades'] if row['str_codigo'] in ['ES','MES@CME'] else 0,axis=1)

off_fut['final'] = off_fut['dbl_lote']+ off_fut['trades']
off_fut['str_codigo']='MES@CME'
off_fut=off_fut[off_fut['dbl_lote']!=0]

off_fut=off_fut.groupby('str_fundo').agg({'dbl_lote':sum,'trades':sum,'final':sum})
position_B=-off_fut.loc['KAPITALO CLASS B','final']*2
off_fut

,dbl_lote,trades,final
str_fundo,,,
KAPITALO CLASS B,15950.0,-10.0,15940.0
KAPITALO CLASS Q,828.0,0.0,828.0


In [9]:
def prop_classB(dt):
    dt = dt.strftime('%Y-%m-%d')
    db_conn_test = psycopg2.connect(
    host="PGKPTL01", dbname="db_Teste", user="kapitalo11", password="kapitalo11"
    )
    query = f"""SELECT * \
        FROM public.tbl_proporcao_fundos where dte_data='{dt}' and  str_tipo='Com Master 1' order by dbl_proporcao"""
    df = pd.read_sql(query, db_conn_test)
    
    names = {
    "ALPHA GLOBAL": "KAPITALO ALPHA GLOBAL MASTER FIM IE",
    "KAPPA PREV": "KAPITALO KAPPA PREV MASTER FIM",
    "SIGMA LLC": "KAPITALO SIGMA LLC",
    "MASTER II": "KAPITALO ZETA MASTER FIM",
    "MASTER 2": "KAPITALO ZETA MASTER FIM",
    "ZETA MASTER FIA": 'KAPITALO ZETA MASTER FIA',
    "MASTER 1": "KAPITALO KAPPA MASTER FIM",
    "KAPPA PREV MASTER": "KAPITALO KAPPA PREV MASTER FIM",
    "K10 PREV MASTER": "KAPITALO K10 PREV MASTER FIM",


    }
    df["str_fundo"] = df["str_fundo"].map(names)

    return df[['str_fundo','dbl_proporcao']].set_index('str_fundo')
    

In [10]:
prop_classB_p=prop_classB(dt)


In [11]:
## CLASS B
prop_classB_p=prop_classB(dt)
df_isp=get_posicao(mercado='Futuro',codigo='ISP',dt=dt_1,serie='H23')
df_wsp=get_posicao(mercado='Futuro',codigo='WSP',dt=dt_1,serie='H23')
# display('1',df_wsp)
df_snp=pd.merge(df_isp,df_wsp,how='outer')

df_snp=df_snp[df_snp['str_codigo'].isin(['ISP','WSP'])]

df_snp.rename(columns={'net_position':'dbl_qtde'},inplace=True)
df_snp['dbl_qtde']=df_snp.apply(lambda row: (row['dbl_qtde']*20) if row['str_codigo']=='ISP' else row['dbl_qtde'] if row['str_codigo'] in ['ISP','WSP'] else 0,axis=1)
df_snp['trades']=df_snp.apply(lambda row: (row['trades']*20) if row['str_codigo']=='ISP' else row['trades'] if row['str_codigo'] in ['ISP','WSP'] else 0,axis=1)
df_snp['str_codigo']='WSP'

df_snp=df_snp.groupby(['str_fundo','str_serie'], as_index=False).agg({'dbl_qtde':sum,'trades':sum})
df_snp=df_snp[df_snp['str_fundo'].isin(['KAPITALO KAPPA MASTER FIM','KAPITALO K10 PREV MASTER FIM','KAPITALO KAPPA PREV MASTER FIM'])]
df_snp['ajuste']=df_snp.apply(lambda row: 100*prop_classB_p.loc[row['str_fundo'],'dbl_proporcao'],axis=1)
df_snp['ideal']=round(((position_B)*df_snp['ajuste']/100),0)
df_snp['dbl_qtde']=df_snp.apply(lambda row: row['dbl_qtde'] if row['str_fundo']=='KAPITALO KAPPA PREV MASTER FIM' else row['dbl_qtde'] ,axis=1)
# df_snp['ajuste']=df_snp['ideal']-df_snp['dbl_qtde']
contratos_wsp=df_snp['ideal'].sum()
df_snp['qtde']= df_snp['ideal']-( df_snp['dbl_qtde']+df_snp['trades'] )
display(df_snp)

# df=df_snp[['str_fundo','trade']]
# df['qtde']=df['trade'].apply(lambda x: int(x) if abs(x)>1 else 0 )
# df['qtde']=df['trade']
# df['MESA'] = 'Kapitalo 11.1'
# df['CLEARING'] = 'BRADESCO'
# df['TIPO']='Futuro'
# df['codigo']='WSP'
# df['str_serie']='U22'
# df['CONTRA'] = 'Bradesco'
# df['ESTRATEGIA'] = 'Arbitragem'
# df['price']= float('3770')
    

# df=df[["str_fundo","MESA","ESTRATEGIA","CLEARING","CONTRA","TIPO", "codigo","str_serie","qtde","price"]]
# df.to_clipboard(excel=True)

,str_fundo,str_serie,dbl_qtde,trades,ajuste,ideal,qtde
1,KAPITALO K10 PREV MASTER FIM,H23,-4071.0,0.0,12.793695,-4079.0,-8.0
2,KAPITALO KAPPA MASTER FIM,H23,-25775.0,20.0,80.770283,-25750.0,5.0
3,KAPITALO KAPPA PREV MASTER FIM,H23,-2032.0,0.0,6.436021,-2052.0,-20.0


## Controle ADR-Local

In [125]:
adr_local = get_carteira_estrategia(dt_1,'Arbitragem Aluguel')[['str_fundo','str_mercado','str_codigo','str_serie','dbl_lote','str_estrategia']]

In [126]:
parse = {
    'CBD': 'PCAR',
    'ABEV':'ABEV',
    'BRFS': 'BRFS',
    'VALE':'VALE',
    'SUZ': 'SUZB'
}

In [127]:
adr = adr_local[adr_local['str_fundo']=='KAPITALO CLASS B']

adr['str_codigo'] = adr['str_codigo'].map(parse)

C:\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [128]:
adr

,str_fundo,str_mercado,str_codigo,str_serie,dbl_lote,str_estrategia
0,KAPITALO CLASS B,Acao,ABEV,ABEV US EQUITY,567400.0,Arbitragem Aluguel
1,KAPITALO CLASS B,Acao,BRFS,BRFS US EQUITY,671320.0,Arbitragem Aluguel
2,KAPITALO CLASS B,Acao,PCAR,CBD US EQUITY,854580.0,Arbitragem Aluguel
3,KAPITALO CLASS B,Acao,SUZB,SUZ US EQUITY,561210.0,Arbitragem Aluguel
4,KAPITALO CLASS B,Acao,VALE,VALE US EQUITY,58070.0,Arbitragem Aluguel


In [129]:
local = adr_local.loc[(adr_local['str_fundo'] !='KAPITALO CLASS B') & (adr_local['str_codigo'] !='WDO')& (adr_local['str_codigo'] !='DOL') ]


In [130]:
local

,str_fundo,str_mercado,str_codigo,str_serie,dbl_lote,str_estrategia
5,KAPITALO K10 PREV MASTER FIM,Acao,ABEV,ABEV3 BZ EQUITY,-56879.0,Arbitragem Aluguel
6,KAPITALO K10 PREV MASTER FIM,Acao,BRFS,BRFS3 BZ EQUITY,-68330.0,Arbitragem Aluguel
7,KAPITALO K10 PREV MASTER FIM,Acao,PCAR,PCAR3 BZ EQUITY,-85598.0,Arbitragem Aluguel
8,KAPITALO K10 PREV MASTER FIM,Acao,SUZB,SUZB3 BZ EQUITY,-55353.0,Arbitragem Aluguel
9,KAPITALO K10 PREV MASTER FIM,Acao,VALE,VALE3 BZ EQUITY,-5903.0,Arbitragem Aluguel
11,KAPITALO KAPPA MASTER FIM,Acao,ABEV,ABEV3 BZ EQUITY,-478640.0,Arbitragem Aluguel
12,KAPITALO KAPPA MASTER FIM,Acao,BRFS,BRFS3 BZ EQUITY,-564882.0,Arbitragem Aluguel
13,KAPITALO KAPPA MASTER FIM,Acao,PCAR,PCAR3 BZ EQUITY,-720988.0,Arbitragem Aluguel
14,KAPITALO KAPPA MASTER FIM,Acao,SUZB,SUZB3 BZ EQUITY,-474870.0,Arbitragem Aluguel
15,KAPITALO KAPPA MASTER FIM,Acao,VALE,VALE3 BZ EQUITY,-48836.0,Arbitragem Aluguel


In [131]:
hedge = adr_local.loc[(adr_local['str_codigo'] =='WDO') |(adr_local['str_codigo'] =='DOL') ]

In [132]:
hedge

,str_fundo,str_mercado,str_codigo,str_serie,dbl_lote,str_estrategia
10,KAPITALO K10 PREV MASTER FIM,Futuro,WDO,X22,128.0,Arbitragem Aluguel
16,KAPITALO KAPPA MASTER FIM,Futuro,DOL,X22,208.0,Arbitragem Aluguel
17,KAPITALO KAPPA MASTER FIM,Futuro,WDO,X22,-2.0,Arbitragem Aluguel
23,KAPITALO KAPPA PREV MASTER FIM,Futuro,DOL,X22,13.0,Arbitragem Aluguel
24,KAPITALO KAPPA PREV MASTER FIM,Futuro,WDO,X22,7.0,Arbitragem Aluguel


In [133]:
adr = adr.set_index('str_codigo')[['dbl_lote']]


In [134]:
adr

,dbl_lote
str_codigo,
ABEV,567400.0
BRFS,671320.0
PCAR,854580.0
SUZB,561210.0
VALE,58070.0


In [135]:
new_trades = {
    'BRFS':0,
    'ABEV':0,
    'PCAR':0,
    'VALE':0,
    'SUZB':0
    
}

In [136]:
local['dbl_lote_ideal'] = local.apply(lambda row: -round( (new_trades[row['str_codigo']] + adr.loc[row['str_codigo'], 'dbl_lote']) *prop_classB_p.loc[row['str_fundo'],'dbl_proporcao'],0) ,axis=1 )
local['trade'] =  local['dbl_lote_ideal']-local['dbl_lote']

C:\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [137]:
local

,str_fundo,str_mercado,str_codigo,str_serie,dbl_lote,str_estrategia,dbl_lote_ideal,trade
5,KAPITALO K10 PREV MASTER FIM,Acao,ABEV,ABEV3 BZ EQUITY,-56879.0,Arbitragem Aluguel,-58616.0,-1737.0
6,KAPITALO K10 PREV MASTER FIM,Acao,BRFS,BRFS3 BZ EQUITY,-68330.0,Arbitragem Aluguel,-69352.0,-1022.0
7,KAPITALO K10 PREV MASTER FIM,Acao,PCAR,PCAR3 BZ EQUITY,-85598.0,Arbitragem Aluguel,-88284.0,-2686.0
8,KAPITALO K10 PREV MASTER FIM,Acao,SUZB,SUZB3 BZ EQUITY,-55353.0,Arbitragem Aluguel,-57977.0,-2624.0
9,KAPITALO K10 PREV MASTER FIM,Acao,VALE,VALE3 BZ EQUITY,-5903.0,Arbitragem Aluguel,-5999.0,-96.0
11,KAPITALO KAPPA MASTER FIM,Acao,ABEV,ABEV3 BZ EQUITY,-478640.0,Arbitragem Aluguel,-475886.0,2754.0
12,KAPITALO KAPPA MASTER FIM,Acao,BRFS,BRFS3 BZ EQUITY,-564882.0,Arbitragem Aluguel,-563045.0,1837.0
13,KAPITALO KAPPA MASTER FIM,Acao,PCAR,PCAR3 BZ EQUITY,-720988.0,Arbitragem Aluguel,-716748.0,4240.0
14,KAPITALO KAPPA MASTER FIM,Acao,SUZB,SUZB3 BZ EQUITY,-474870.0,Arbitragem Aluguel,-470694.0,4176.0
15,KAPITALO KAPPA MASTER FIM,Acao,VALE,VALE3 BZ EQUITY,-48836.0,Arbitragem Aluguel,-48704.0,132.0


In [138]:
position =  local[['str_serie','dbl_lote']].groupby(by=['str_serie']).sum()

In [139]:
position

,dbl_lote
str_serie,
ABEV3 BZ EQUITY,-567400.0
BRFS3 BZ EQUITY,-671300.0
PCAR3 BZ EQUITY,-854700.0
SUZB3 BZ EQUITY,-561200.0
VALE3 BZ EQUITY,-58030.0


In [140]:
position.to_clipboard(excel=True)

In [141]:
alloc = local.groupby(by=['str_codigo','str_fundo']).sum()

In [97]:
alloc.to_clipboard()

In [142]:
alloc

dbl_lote  dbl_lote_ideal   trade
str_codigo str_fundo                                                       
ABEV       KAPITALO K10 PREV MASTER FIM    -56879.0        -58616.0 -1737.0
           KAPITALO KAPPA MASTER FIM      -478640.0       -475886.0  2754.0
           KAPITALO KAPPA PREV MASTER FIM  -31881.0        -32898.0 -1017.0
BRFS       KAPITALO K10 PREV MASTER FIM    -68330.0        -69352.0 -1022.0
           KAPITALO KAPPA MASTER FIM      -564882.0       -563045.0  1837.0
           KAPITALO KAPPA PREV MASTER FIM  -38088.0        -38923.0  -835.0
PCAR       KAPITALO K10 PREV MASTER FIM    -85598.0        -88284.0 -2686.0
           KAPITALO KAPPA MASTER FIM      -720988.0       -716748.0  4240.0
           KAPITALO KAPPA PREV MASTER FIM  -48114.0        -49549.0 -1435.0
SUZB       KAPITALO K10 PREV MASTER FIM    -55353.0        -57977.0 -2624.0
           KAPITALO KAPPA MASTER FIM      -474870.0       -470694.0  4176.0
           KAPITALO KAPPA PREV MASTER FIM  -30977.0        -32539.0 -1562.0
VALE       KAPITALO K10 PREV MASTER FIM     -5903.0         -5999.0   -96.0
           KAPITALO KAPPA MASTER FIM       -48836.0        -48704.0   132.0
           KAPITALO KAPPA PREV MASTER FIM   -3291.0         -3367.0   -76.0

In [87]:
adr_prop = dict()
prop_trade = dict()

In [88]:
for key,value in adr_prop.items():
    adr_prop[key] = alloc.loc[key]['trade'].to_dict()

In [89]:
for i,row in alloc.iterrows():
    adr_prop[i[0]] = alloc.loc[i[0]]['trade'].to_dict()
    

In [ ]:
adr_prop

In [ ]:
sum([abs(x) for x in adr_prop['ABEV'].values()])

In [ ]:
for i,key in adr_prop.items():
    total = sum([abs(x) for x in adr_prop[i].values()])
    for i2,key2 in key.items():
        adr_prop[i][i2] = key2/total

In [ ]:
adr_prop

In [ ]:
trades = 0

In [ ]:
hedge['dbl_lote_ideal'] = hedge.apply(lambda row: round((hedge['dbl_lote'].sum()+trades)*prop_classB_p.loc[row['str_fundo'],'dbl_proporcao'],0),axis=1 )
hedge ['trade'] =  hedge['dbl_lote_ideal']-hedge['dbl_lote']

In [ ]:
hedge

# Analise Spread


In [8]:
query_on= "select dte_data,str_codigo,str_serie, sum(dbl_lote), AVG(dbl_preco) from tbl_auxboletas1 where str_estrategia='Arbitragem Aluguel' and str_mesa='Kapitalo 11.1' and dte_data>'2022-07-01' and str_codigo <> 'WDO' group by str_codigo,str_serie,dte_data HAVING sum(dbl_lote)<>0"
query_dol = "select dte_data,str_codigo,str_serie, sum(dbl_lote), AVG(dbl_preco) from tbl_auxboletas1 where str_estrategia='Arbitragem Aluguel' and str_mesa='Kapitalo 11.1' and dte_data>'2022-07-01' and str_codigo = 'WDO' group by str_codigo,str_serie,dte_data HAVING sum(dbl_lote)<>0"
query_off = "select dte_data,str_codigo,str_serie, sum(dbl_qtetrade), AVG(dbl_putrade) from tbl_boletasexternas where str_estrategia='Arbitragem Aluguel' and str_mesa='Kapitalo 11.1' and dte_data>'2022-07-01' group by str_codigo,str_serie,dte_data"
db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)
on = pd.read_sql(query_on,db_conn_test)

off = pd.read_sql(query_off,db_conn_test).rename(columns={'str_serie':'str_serie_off','sum':'size','avg':'price'})
dol = pd.read_sql(query_dol,db_conn_test)
db_conn_test.close()

fee = 0.0032

Devol =98.5/100

In [9]:
off['abs_size'] = abs(off['size'])

In [10]:
on

,dte_data,str_codigo,str_serie,sum,avg
0,2022-08-01,ABEV,ABEV3 BZ EQUITY,-700.0,14.620000
1,2022-08-04,ABEV,ABEV3 BZ EQUITY,-348500.0,14.605833
2,2022-08-05,ABEV,ABEV3 BZ EQUITY,-203800.0,14.656698
3,2022-08-08,ABEV,ABEV3 BZ EQUITY,-13500.0,14.609407
4,2022-08-10,ABEV,ABEV3 BZ EQUITY,-900.0,14.886667
5,2022-07-29,BRFS,BRFS3 BZ EQUITY,-121600.0,15.848235
6,2022-08-01,BRFS,BRFS3 BZ EQUITY,-497300.0,16.640000
7,2022-08-02,BRFS,BRFS3 BZ EQUITY,-305000.0,16.477143
8,2022-08-05,BRFS,BRFS3 BZ EQUITY,-300.0,16.820000
9,2022-08-30,DR1,U2V2,142.0,39.550000


In [17]:
# Fechamentos do Dolar
db_conn_risk = psycopg2.connect(host=config.DB_RISK_HOST, dbname=config.DB_RISK_NAME , user=config.DB_RISK_USER, password=config.DB_RISK_PASS)
query_risk = "SELECT rptdt, tckrsymb, adjstdqt FROM b3up2data.currency_eod_settlementpricefile_futures where tckrsymb like '%WDO%'"

adj_dol= pd.read_sql(query_risk,db_conn_risk)

In [18]:
adj_dol

,rptdt,tckrsymb,adjstdqt
0,2019-04-18,WDOF20,3989.779
1,2019-04-18,WDOF21,4131.890
2,2019-04-18,WDOF25,5176.129
3,2019-04-18,WDOJ20,4015.544
4,2019-04-18,WDOJ22,4412.244
...,...,...,...
11226,2022-09-30,WDOK23,5600.804
11227,2022-09-30,WDOV22,5406.600
11228,2022-09-30,WDOV23,5751.463
11229,2022-09-30,WDOX22,5403.660


In [19]:
off['str_codigo'] =off['str_codigo'].map(parse)

In [20]:
on['abs_size'] = abs(on['sum'])

In [21]:
adr_local = on.merge(off,on=['str_codigo','abs_size'],how='left')

In [22]:
adr_local

,dte_data_x,str_codigo,str_serie,sum,avg,abs_size,dte_data_y,str_serie_off,size,price
0,2022-08-01,ABEV,ABEV3 BZ EQUITY,-700.0,14.620000,700.0,2022-08-01,ABEV US EQUITY,700.0,2.780000
1,2022-08-04,ABEV,ABEV3 BZ EQUITY,-348500.0,14.605833,348500.0,2022-08-04,ABEV US EQUITY,348500.0,2.755759
2,2022-08-05,ABEV,ABEV3 BZ EQUITY,-203800.0,14.656698,203800.0,2022-08-05,ABEV US EQUITY,203800.0,2.782614
3,2022-08-08,ABEV,ABEV3 BZ EQUITY,-13500.0,14.609407,13500.0,2022-08-08,ABEV US EQUITY,13500.0,2.807778
4,2022-08-10,ABEV,ABEV3 BZ EQUITY,-900.0,14.886667,900.0,2022-08-10,ABEV US EQUITY,900.0,2.908500
5,2022-07-29,BRFS,BRFS3 BZ EQUITY,-121600.0,15.848235,121600.0,2022-07-29,BRFS US EQUITY,121600.0,3.001201
6,2022-08-01,BRFS,BRFS3 BZ EQUITY,-497300.0,16.640000,497300.0,2022-08-01,BRFS US EQUITY,497300.0,3.176965
7,2022-08-02,BRFS,BRFS3 BZ EQUITY,-305000.0,16.477143,305000.0,2022-08-02,BRFS US EQUITY,305000.0,3.080904
8,2022-08-05,BRFS,BRFS3 BZ EQUITY,-300.0,16.820000,300.0,2022-08-05,BRFS US EQUITY,300.0,3.210000
9,2022-08-30,DR1,U2V2,142.0,39.550000,142.0,NaN,NaN,NaN,NaN


In [23]:
dates = (adr_local['dte_data_y'].tolist() +adr_local['dte_data_x'].tolist())

In [24]:
query_c = "select dte_data, dbl_preco from tbl_indicadores1 where str_indicador='CASADO Curncy'"
db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)
casado =pd.read_sql(query_c,db_conn_test)
db_conn_test.close()

In [25]:
casado=casado[casado['dte_data'].isin(dates)]

In [26]:
casado

,dte_data,dbl_preco
3604,2022-07-29,48.60
3606,2022-08-01,47.35
3607,2022-08-02,44.75
3609,2022-08-04,40.60
3610,2022-08-05,38.65
3611,2022-09-22,9.10
3612,2022-08-08,36.00
3614,2022-08-09,33.10
3615,2022-08-10,30.65
3629,2022-09-05,35.40


In [27]:
dol=casado.merge(dol,on='dte_data',how='left')
dol['str_codigo'] =dol['str_codigo'].iloc[0]  

dol['str_serie'] =dol['str_serie'].iloc[0] 

ticker = dol['str_codigo'].iloc[0]  + dol['str_serie'].iloc[0] 
dol = dol.fillna(0)

dol.loc[dol['avg']==0.0,'avg'] = adj_dol.loc[(adj_dol['tckrsymb']==ticker) 
& (adj_dol['rptdt']== dol.loc[dol['avg']==0.0,'dte_data'].values[0] ) ,'adjstdqt'].values[0] 
dol['spot'] = (dol['avg']-dol['dbl_preco'])/1000
dol=dol[['dte_data','spot']]

In [28]:
adr_local = adr_local.rename(columns={'dte_data_y':'dte_data'})
adr_local=adr_local.merge(dol,on='dte_data')

In [30]:
adr_local['spread'] = ((adr_local['price']*adr_local['spot'])/adr_local['avg'] -1)*10000


In [31]:
adr_local['custo off bps'] = ((adr_local['price']-fee)/adr_local['price'] -1)* 10000

adr_local['custo on bps'] = -(0.023/100 +0.5/100 * (1-Devol))* 10000

In [32]:
adr_local

,dte_data_x,str_codigo,str_serie,sum,avg,abs_size,dte_data,str_serie_off,size,price,spot,spread,custo off bps,custo on bps
0,2022-08-01,ABEV,ABEV3 BZ EQUITY,-700.0,14.620000,700.0,2022-08-01,ABEV US EQUITY,700.0,2.780000,5.176671,-156.534782,-11.510791,-3.05
1,2022-08-01,BRFS,BRFS3 BZ EQUITY,-497300.0,16.640000,497300.0,2022-08-01,BRFS US EQUITY,497300.0,3.176965,5.176671,-116.524362,-10.072506,-3.05
2,2022-08-04,ABEV,ABEV3 BZ EQUITY,-348500.0,14.605833,348500.0,2022-08-04,ABEV US EQUITY,348500.0,2.755759,5.222944,-145.598530,-11.612046,-3.05
3,2022-08-05,ABEV,ABEV3 BZ EQUITY,-203800.0,14.656698,203800.0,2022-08-05,ABEV US EQUITY,203800.0,2.782614,5.174540,-175.995652,-11.499978,-3.05
4,2022-08-05,BRFS,BRFS3 BZ EQUITY,-300.0,16.820000,300.0,2022-08-05,BRFS US EQUITY,300.0,3.210000,5.174540,-124.689481,-9.968847,-3.05
5,2022-08-05,PCAR,PCAR3 BZ EQUITY,-63600.0,16.452013,63600.0,2022-08-05,CBD US EQUITY,63600.0,3.138821,5.174540,-127.679725,-10.194911,-3.05
6,2022-08-08,ABEV,ABEV3 BZ EQUITY,-13500.0,14.609407,13500.0,2022-08-08,ABEV US EQUITY,13500.0,2.807778,5.118973,-161.860090,-11.396912,-3.05
7,2022-08-08,PCAR,PCAR3 BZ EQUITY,-432600.0,16.908350,432600.0,2022-08-08,CBD US EQUITY,432600.0,3.262298,5.118973,-123.450857,-9.809036,-3.05
8,2022-08-10,ABEV,ABEV3 BZ EQUITY,-900.0,14.886667,900.0,2022-08-10,ABEV US EQUITY,900.0,2.908500,5.092082,-51.284933,-11.002235,-3.05
9,2022-07-29,BRFS,BRFS3 BZ EQUITY,-121600.0,15.848235,121600.0,2022-07-29,BRFS US EQUITY,121600.0,3.001201,5.191295,-169.177409,-10.662398,-3.05


In [33]:
adr_local['product'] = (adr_local['spread'] -2*adr_local['custo off bps'] - 2*adr_local['custo on bps'])* adr_local['size']

In [34]:
aux = adr_local.groupby(['str_codigo']).agg({'size':sum,'product':sum})

In [35]:
aux['medio'] = (aux['product']/aux['size'])
# aux[['medio']].to_clipboard(excel='True')

In [36]:
aux[['size','medio']].loc[aux['size']!=0]

,size,medio
str_codigo,,
ABEV,567400.0,-127.536272
BRFS,924200.0,-117.564526
PCAR,496200.0,-98.175898
SUZB,503600.0,-47.663396
VALE,49300.0,-1.188255


In [ ]:
import numpy as np

In [ ]:
adr_local

In [ ]:
db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)

In [ ]:
query = "select * from tbl_auxboletas1 where str_estrategia='Box_3pontas' and dte_data>'2022-08-15' and str_codigo='IRBR'"

In [ ]:
box = pd.read_sql(query,db_conn_test)[['str_codigo','str_serie','dbl_lote','dbl_preco']]

In [ ]:
spot = box[box['str_serie']=='IRBR3 BZ EQUITY'].loc[box['dbl_lote']<0]

In [ ]:
spot['sumproduct']= spot['dbl_lote']*spot['dbl_preco']
spot = spot.groupby(['str_serie']).agg({'dbl_lote':sum,'sumproduct':sum}).reset_index()
spot['médio'] = spot['sumproduct']/spot['dbl_lote']

In [ ]:
spot

In [ ]:
box['sumproduct']= box['dbl_lote']*box['dbl_preco']
box = box.groupby(['str_serie']).agg({'dbl_lote':sum,'sumproduct':sum}).reset_index()
box['médio'] = box['sumproduct']/box['dbl_lote']

In [ ]:
box

In [ ]:
synth = box[box['str_serie']!='IRBR3 BZ EQUITY']

In [ ]:
synth['strike'] = synth['str_serie'].apply(lambda x: x[-3:]) 

In [ ]:
synth_call = synth[synth['dbl_lote']>0].rename(columns={'médio':'call'})
synth_put = synth[synth['dbl_lote']<0].rename(columns={'médio':'put'})

In [ ]:
df = synth_call.merge(synth_put,on='strike',how = 'left')

In [ ]:
df['synt spread'] = df['call'] - df['put']

In [ ]:
df['box_spread'] = df['synt spread'] - spot.iloc[0]['médio']

In [ ]:
df

In [ ]:
db_conn_test = psycopg2.connect(host=config.DB_TESTE_HOST, dbname=config.DB_TESTE_NAME , user=config.DB_TESTE_USER, password=config.DB_TESTE_PASS)

In [ ]:
query = "select * from tbl_gerencialvalorizacao where str_mesa='Kapitalo 11.1' and str_estrategia='Bolsa' and str_fundo = 'KAPITALO KAPPA MASTER FIM' and str_codigo='Total Aluguel'"

In [ ]:
df = pd.read_sql(query,db_conn_test)

In [116]:



ref  = datetime.date(2022,10,10)

In [117]:
workdays.networkdays(ref,datetime.date(2022,12,26),holidays_br)


52

In [ ]:
workdays.timedelta()

In [114]:
52*8+31*8

664

In [108]:
244/8

30.5

In [115]:
workdays.networkdays(datetime.date(2023,2,1),datetime.date(2023,3,20),holidays_br)

31